In [2]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool


llm = ChatOllama(model='gemma3:1b')
llm.invoke('Привет')

AIMessage(content='Привет! Как дела? Чем занимаешься? 😊\n', additional_kwargs={}, response_metadata={'model': 'gemma3:1b', 'created_at': '2025-05-17T11:35:35.609333Z', 'done': True, 'done_reason': 'stop', 'total_duration': 359675875, 'load_duration': 46547292, 'prompt_eval_count': 10, 'prompt_eval_duration': 69082584, 'eval_count': 12, 'eval_duration': 243412833, 'model_name': 'gemma3:1b'}, id='run--f9daa3f3-f1aa-4f82-913b-ec520cbd9bfc-0', usage_metadata={'input_tokens': 10, 'output_tokens': 12, 'total_tokens': 22})

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///example.db")

In [6]:
db.dialect
db.get_usable_table_names()

['transactions']

In [8]:
db.run("SELECT * FROM transactions LIMIT 5")

"[(1, 'user_1', 'expense', 'макдональдс', 879.0, '2025-05-11'), (2, 'user_1', 'expense', 'макдональдс', 879.0, '2025-05-11')]"

### SQL Query Chain

In [12]:
from langchain.chains import create_sql_query_chain

sql_chain = create_sql_query_chain(llm, db)

In [13]:
sql_chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [14]:
sql_chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.\nNever query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.\nPay attention to use only the column names you can see in the

In [17]:
question = "Сколько я потратил на макдональдс?"
response = sql_chain.invoke({'question': question})
print(response)

SELECT amount FROM transactions WHERE type = 'expense' LIMIT 5


In [10]:
from ask_llm import ask_llm
from langchain_core.runnables import chain

In [11]:
@chain
def get_correct_sql_query(input):
    context = input['context']
    question = input['question']

    instruction = """
        Используй приведённый выше контекст, чтобы составить корректный SQL-запрос для следующего вопроса:
        {}

        НЕ заключай запрос в блок ```sql и НЕ добавляй вступление или пояснения.
        ОБЯЗАТЕЛЬНО верни только ОДИН SQL-запрос.
    """.format(question)

    response = ask_llm(context=context, question=instruction)

    return response

In [ ]:
response = get_correct_sql_query.invoke({'context': response, 'question': question})